In [ ]:
!nvidia-smi

Fri Aug 18 16:34:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 19.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Adult-Census
data = pd.read_csv("https://raw.githubusercontent.com/pooja2512/Adult-Census-Income/master/adult.csv")
data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [ ]:
(data["workclass"] == "?").any()

True

In [ ]:
columns_with_q_mark = [col for col in data.columns if (data[col] == "?").any()]
columns_with_q_mark

['workclass', 'occupation', 'native.country']

In [ ]:
for col in columns_with_q_mark:
    data[col] = data[col].replace("?", "Unknown")

data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,Unknown,77053,HS-grad,9,Widowed,Unknown,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,Unknown,186061,Some-college,10,Widowed,Unknown,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [ ]:
data["income"].value_counts(normalize = True) * 100

<=50K    75.919044
>50K     24.080956
Name: income, dtype: float64

In [ ]:
X, y = data.drop(["income"], axis = 1), data["income"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2,
    random_state = 34589,
    stratify = y
)

In [ ]:
X_train.shape

(26048, 14)

In [ ]:
y_train

6647     <=50K
29774    <=50K
7340     <=50K
9052     <=50K
24146    <=50K
         ...  
24449     >50K
27376    <=50K
12036     >50K
31285    <=50K
3288     <=50K
Name: income, Length: 26048, dtype: object

--> Kategorik sütunları nümerik hale getiriyorduk ama CatBoost için bunu direkt olarak yapmak zorunda değiliz.

In [ ]:
X_train

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
6647,25,Private,216010,Assoc-acdm,12,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,40,United-States
29774,27,Self-emp-not-inc,357283,HS-grad,9,Never-married,Sales,Not-in-family,Black,Male,0,0,40,United-States
7340,25,Local-gov,44216,HS-grad,9,Divorced,Adm-clerical,Not-in-family,Amer-Indian-Eskimo,Female,0,0,35,United-States
9052,38,Local-gov,134444,Some-college,10,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,72,United-States
24146,37,Private,83893,Assoc-voc,11,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24449,51,Federal-gov,140516,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States
27376,60,Self-emp-not-inc,21101,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,Amer-Indian-Eskimo,Male,0,0,50,United-States
12036,34,Self-emp-not-inc,204375,Prof-school,15,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,60,United-States
31285,38,Private,54317,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,60,United-States


In [ ]:
categorical_features = list(X_train.select_dtypes(include = "object").columns)
categorical_features

['workclass',
 'education',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native.country']

In [ ]:
# ML-DL kütüphanelerinde buna Data API denir. #
# Pool'dan bir tane veri aldığınız zaman
# --> feature, label şeklinde dönecek.
train_pool = Pool(X_train, y_train, cat_features = categorical_features)
test_pool = Pool(X_test, y_test, cat_features = categorical_features)

In [ ]:
"""
XGBoost, LightGBM
CatBoost --> Ordering Boosting diye bir method kullanılır.
            - Overfittinge karşı bir tık dayanıklı.

            Overfitting (Aşırı Uyum): ML modelimizin train ettiğimiz verisetinde
            çok iyi performans gösterip, hiç görmediği test veya gelecek olan
            verisetinde kötü performans göstermesidir. Modelimizin bir nevi
            train verisetini ezberlemesidir.

            Bizim ML'deki amacımız train verisetini genellemektedir.
"""
model = CatBoostClassifier(n_estimators = 300,
                           task_type = "GPU")

In [ ]:
param_dist = {
    "learning_rate": [0.03, 0.1, 0.3],
    "depth": [3, 5, 7],
    "l2_leaf_reg": [1, 3, 5] # l2_leaf_regularization
}

In [ ]:
search_results = model.randomized_search(
                    param_dist, train_pool,
                    cv = 3, n_iter = 10,
                    partition_random_seed = 548574,
                    verbose = 1)

0:	learn: 0.4804988	test: 0.4804323	best: 0.4804323 (0)	total: 22.8ms	remaining: 6.8s
1:	learn: 0.4102100	test: 0.4102592	best: 0.4102592 (1)	total: 42.1ms	remaining: 6.27s
2:	learn: 0.3751383	test: 0.3758885	best: 0.3758885 (2)	total: 61.3ms	remaining: 6.07s
3:	learn: 0.3512458	test: 0.3511141	best: 0.3511141 (3)	total: 82.4ms	remaining: 6.1s
4:	learn: 0.3403176	test: 0.3403817	best: 0.3403817 (4)	total: 103ms	remaining: 6.1s
5:	learn: 0.3317685	test: 0.3317538	best: 0.3317538 (5)	total: 124ms	remaining: 6.07s
6:	learn: 0.3258418	test: 0.3264275	best: 0.3264275 (6)	total: 145ms	remaining: 6.08s
7:	learn: 0.3207672	test: 0.3216888	best: 0.3216888 (7)	total: 164ms	remaining: 5.99s
8:	learn: 0.3185425	test: 0.3195656	best: 0.3195656 (8)	total: 182ms	remaining: 5.89s
9:	learn: 0.3156246	test: 0.3161867	best: 0.3161867 (9)	total: 201ms	remaining: 5.84s
10:	learn: 0.3123519	test: 0.3136092	best: 0.3136092 (10)	total: 221ms	remaining: 5.8s
11:	learn: 0.3104949	test: 0.3120249	best: 0.3120249

In [ ]:
search_results["params"]

{'depth': 5, 'l2_leaf_reg': 5, 'learning_rate': 0.3}